<a href="https://colab.research.google.com/github/mk-a/NLP_Duty1/blob/master/GRU2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
import pickle
import torch.nn as nn
import time
import gc
import copy

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_df = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/IFT6285/train_post_preprocessed2.pkl")
valid_df = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/IFT6285/test_preprocessed.pkl")

In [5]:
(np.array([len(a) for a in train_df.text.values]) < 300).sum()/len(train_df)

0.7631698557826421

In [0]:
count_words = Counter()
for post in train_df.text.values:
    for tok in post:
        count_words[tok] += 1

In [17]:
len(count_words)

685119

In [18]:
len([w for w in count_words if count_words[w]==1])

408247

In [19]:
len(count_words) - len([w for w in count_words if count_words[w]==1])

276872

In [21]:
len(train_df)

512629

In [0]:
def text2seq(X, y, seq_len, vocab_max=None, word2idx=None ):
    """ For a list of inputs X with different sequence length, associated to the labels y.
        Builds the list of the vocab_max-2 most common words. Maps each of them to a number.
        Replace the other words by the token unk_tok, which is mapped to the value 1.
        Then force the input to have a sequence length of seq_len with this policy:
            - If the sample x is short than seq_len, then adds padding at the begining.
            The padding token is set by the variable padd_tok. And mapped to the value 0.
            - If the sample x is longer than seq_len, then extracts (len(x)//seq_len)+1
            sequences from x.
    """
    if word2idx == None:
        count_words = Counter()
        for post in X:
            for tok in post:
                count_words[tok] += 1
        if vocab_max == None:
            vocab_max = len(count_words) + 2
        vocab = {w for w,_ in count_words.most_common(vocab_max-2)}
        word2idx = {w : i+2 for i, (w,_) in enumerate( count_words.most_common(vocab_max-2) )}
        word2idx["PAD"] = 0
        word2idx["UNK"] = 1
    else:
        vocab = {k for k in word2idx.keys()}

    X2 = []
    y2 = []
    for i, x in enumerate(X):
        if len(x) < seq_len:
            tmp = x.copy()
            for _ in range(seq_len - len(x)):
                tmp.insert(0, 'PAD')
            X2.append([word2idx[tok] if tok in vocab else 1 for tok in tmp])
            y2.append(y[i])
        elif len(x) == seq_len:
            X2.append([word2idx[tok] if tok in vocab else 1 for tok in x]) 
            y2.append(y[i])
        else :
            for j in range(len(x)//seq_len):
                X2.append([word2idx[tok] if tok in vocab else 1 for tok in  x[j*seq_len:(j+1)*seq_len]])
                y2.append(y[i])
            X2.append([word2idx[tok] if tok in vocab else 1 for tok in  x[-seq_len:]])
            y2.append(y[i])
    return word2idx, X2, y2

In [0]:
vocab_max = 276874
word2idx, X_train, y_train = text2seq(train_df.text.values, train_df.label.values, 300, vocab_max) 
_, X_valid, y_valid = text2seq(valid_df.text.values, valid_df.label.values, 300, word2idx=word2idx) 
# del data
# gc.collect()

In [22]:
len(X_train)

705845

In [23]:
# with open('/content/drive/My Drive/Colab Notebooks/IFT6285/word2idx_{}.pkl'.format(vocab_max), 'wb') as handle:
#     pickle.dump(word2idx, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('/content/drive/My Drive/Colab Notebooks/IFT6285/X_v{}_s300.pkl'.format(vocab_max), 'wb') as handle:
#     pickle.dump(X_tra, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('/content/drive/My Drive/Colab Notebooks/IFT6285/y_s300.pkl', 'wb') as handle:
#     pickle.dump(y, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: ignored

In [0]:
# load files
# with open('/content/drive/My Drive/Colab Notebooks/IFT6285/v2/word2val_284467.pkl', 'rb') as handle:
#     word2val = pickle.load(handle)
# with open('/content/drive/My Drive/Colab Notebooks/IFT6285/v2/X_v284467_s200.pkl', 'rb') as handle:
#     X = pickle.load(handle)
# with open('/content/drive/My Drive/Colab Notebooks/IFT6285/v2/y_s200.pkl', 'rb') as handle:
#     y = pickle.load(handle)
# vocab_max = 284467

In [0]:
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)
# X_train, X_valid, y_train, y_valid = train_test_split(X2, y2, test_size=0.33, random_state=42)
# X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=42)

X_train = torch.tensor(X_train)
X_valid = torch.tensor(X_valid)
# X_test = torch.tensor(X_test)

y_train = torch.tensor(y_train)
y_valid = torch.tensor(y_valid)
# y_test = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(X_train, y_train)
valid_data = torch.utils.data.TensorDataset(X_valid, y_valid)
# test_data = torch.utils.data.TensorDataset(X_test, y_test)

In [0]:
batch_size = 256

train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(valid_data, shuffle=True, batch_size=batch_size)
# test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [0]:
class GRU(nn.Module):
    def __init__(self,
                 emb_size,
                 hidden_size,
                 vocab_size,
                 num_layers,
                 output_size,
                 nonlinearity = 'relu',
                 bias = True,
                 dropout = 0,
                 bidirectional = True
                ):
        super(GRU, self).__init__()
        #hyper-parameters
        self.emb_size      = emb_size
        self.hidden_size   = hidden_size
        self.vocab_size    = vocab_size
        self.num_layers    = num_layers
        self.output_size   = output_size
        self.nonlinearity  = nonlinearity
        self.bias          = bias
        self.dropout       = dropout
        self.bidirectional = bidirectional
        #layers
        self.embedding = nn.Embedding(vocab_size, emb_size).to(device)
        self.gru = nn.GRU(input_size = emb_size,
                          hidden_size = hidden_size,
                          num_layers = num_layers,
                          bias = bias,
                          dropout = dropout,
                          bidirectional = bidirectional
                         ).to(device)
        self.linear = nn.Linear((bidirectional+1) *hidden_size, output_size).to(device)
        
    def forward(self, X):
        out = self.embedding(X.t())
        out, _ = self.gru(out)
        return self.linear(out[-1])
    
    def predict(self, data):
        if isinstance(data, torch.utils.data.DataLoader):
            y = []
            for i in range((len(data.dataset)//data.batch_size)+1):
                imin = i*data.batch_size
                imax = min((i+1)*batch_size, len(data.dataset))
                y.append(self.forward(data.dataset[imin:imax][0].to(device)).max(dim=1)[1])
            return torch.cat(y)
        elif isinstance(data, torch.Tensor):
            return self.forward(data.to(device)).max(dim=1)[1]
        elif isinstance(data, torch.utils.data.dataset.TensorDataset):
            return self.predict(data.tensors[0])
        else:
            raise AttributeError("data must be an instance of DataLoader, Tensor or TensorDataset")
        

    def accuracy(self, X, y=None):
        if isinstance(X, torch.utils.data.DataLoader):
            return (self.predict(X) == X.dataset.tensors[1].to(device)).float().sum().item() / len(X.dataset.tensors[1])
        elif isinstance(X, torch.Tensor) and isinstance(y, torch.Tensor):
            return (self.predict(X.to(device)) == y.to(device)).float().sum().item() / len(y)
        else :
            raise AttributeError("Must be called with X as a DataLoader or X and y some Tensor")
    
    def f1_score(self, data_loader, label=1):
        return f1_score((data_loader.dataset[:][1] == label).int().cpu().numpy(),
                        (self.predict(data_loader) == label).int().cpu().numpy() )

    def run_epoch(self, n_epochs, criterion, optimizer, train_loader,
                  valid_loader=None, early_stop=True, patience=1 ):
        start_time = time.time()
        last_time = time.time()
        if valid_loader:
            best_score = -1
            best_param = None
            epochs_since_best_score = 0
        for epoch in range(1, n_epochs+1):
            print("Epoch: {}/{}".format(epoch, n_epochs))
            running_loss = 0
            for i, (X_batch, y_batch) in enumerate(train_loader):
                y_batch = y_batch.to(device)
                torch.cuda.empty_cache()
                optimizer.zero_grad()
                out = self.forward(X_batch.to(device))
                loss = criterion(out, y_batch)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            train_score = self.accuracy(train_loader)
            if valid_loader:
                valid_score = self.accuracy(valid_loader)
                print("loss: {:.6f}\ttrain_acc: {:.4f}\telapsed_time: {:.1f}s\tvalid_acc: {:.4f}".format(
                    running_loss/len(train_loader.dataset), train_score, time.time()-start_time, valid_score))
                if best_score > valid_score:
                    epochs_since_best_score +=1
                    if early_stop and epochs_since_best_score == patience:
                        print("Early stopping. Resetting the parameters of the\
                        best score on the validation set.")
                        self.load_state_dict(best_param)
                        return
                else:
                    best_score  = valid_score
                    best_param = copy.deepcopy(self.state_dict())
                    epochs_since_best_score = 0
            else:
                print("loss: {:.6f}\ttrain_acc: {:.4f}\telapsed_time: {:.1f}s".format(
                    running_loss/len(train_loader.dataset), train_score, time.time()-start_time))

In [0]:
model = 'GRU'
emb_size = 32
hidden_size = 64
num_layers = 4
output_size = 3
bidirectional=False
dropout = 0
filename = '{}{}_emb{}_hid{}_lay{}_vocab{}_dp{}.pt'.format('bi' if bidirectional else '', model, emb_size, hidden_size, num_layers, vocab_max, dropout )


gru = GRU(emb_size = emb_size, hidden_size = hidden_size, vocab_size = vocab_max,
          num_layers = num_layers, output_size = output_size, bidirectional=bidirectional, dropout=dropout)

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru.parameters(), weight_decay=10e-5)

In [0]:
gru.run_epoch(n_epochs=20, criterion=criterion, optimizer=optimizer, train_loader=train_loader, valid_loader=valid_loader, early_stop=True, patience=5)

Epoch: 1/20
loss: 0.003319	train_acc: 0.6411	elapsed_time: 264.5s	valid_acc: 0.6012
Epoch: 2/20
loss: 0.002900	train_acc: 0.6872	elapsed_time: 529.7s	valid_acc: 0.6309
Epoch: 3/20
loss: 0.002694	train_acc: 0.7175	elapsed_time: 795.9s	valid_acc: 0.6337
Epoch: 4/20


In [28]:
path = '/content/drive/My Drive/Colab Notebooks/IFT6285/models'
torch.save(gru, path+filename)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type GRU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [27]:
gru.accuracy(valid_loader)

0.6761395732846652